# Unbiasing COVID-19 Case Count
Kartik Chugh - May 3, 2020

 <img src="china-data.png" />

In [ ]:
import numpy as np
import pandas as pd
from pandas import Timestamp
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
np.set_printoptions(suppress=True,
   formatter={'float_kind':'{:0.3f}'.format})

In [ ]:
ONSET_TO_DEATH = 17 # https://midasnetwork.us/covid-19/
MORTALITY_RATE = 0.01 * 10
#DOUBLING_TIME = 2.5

In [ ]:
url = 'https://covidtracking.com/api/v1/us/daily.csv'
df = pd.read_csv(url, 
            usecols=['date', 'positiveIncrease', 'deathIncrease', 'positive'], 
            parse_dates=['date'], 
            index_col=['date']).sort_index().iloc[31:]

time = df.index
positive = np.array(df['positive'])

In [ ]:
pd.set_option('display.max_rows', None)
df

In [ ]:
def doublingTime(a, b):
    return np.log(2)/np.log(b/a) if (a != b) else NaN

doubling = np.empty(positive.shape)
for t_2 in range(1, len(doubling)):
    doubling[t_2] = doublingTime(positive[t_2-1], positive[t_2])
    
doubling_orig = pd.DataFrame(doubling, columns=['doubling'])
doubling_orig = doubling_orig.set_index(time)
    
doubling_smooth = doubling_orig.rolling(7,
        win_type='gaussian',
        center=True).mean(std=2).round()

In [ ]:
plt.plot(doubling_orig)
plt.plot(doubling_smooth)

In [ ]:
doubling_orig.plot(title='Coronavirus Doubling Time',
                   c='k',
                   linestyle=':',
                   alpha=.5,
                   label='Actual',
                   legend=True)

ax = doubling_smooth.plot(label='Smoothed',
                   legend=True)

ax.get_figure().set_facecolor('w')

In [ ]:
firstDeathDate = data['deathIncrease'].ne(0).idxmax()
#firstDeathDate = Timestamp('2020-03-15')
firstDeathDate

In [ ]:
daysLeft = (data.index[-1] - firstDeathDate).days
daysLeft

In [ ]:
firstDeathCount = data['deathIncrease'][firstDeathDate]
firstDeathCount

In [ ]:
firstTrueCaseDate = firstDeathDate - timedelta(days = ONSET_TO_DEATH)
firstTrueCaseDate

In [ ]:
firstTrueCaseCount = firstDeathCount / MORTALITY_RATE
firstTrueCaseCount

In [ ]:
trueCaseCounts = np.logspace(1/DOUBLING_TIME, daysLeft/DOUBLING_TIME, base=2, num=daysLeft)*firstTrueCaseCount

In [ ]:
new = pd.DataFrame(data=trueCaseCounts, index=data.index[-daysLeft:])

In [ ]:
plt.plot(data)